# Election Statistics ANALYSIS

## Importing Libraries

In [46]:
from pandas import DataFrame,read_csv,concat,options,set_option,to_datetime,to_numeric
from os import listdir

In [63]:
options.display.float_format = "{:,.2f}".format
set_option('display.max_rows',None)

## National/State Summaries

### Load Data

In [30]:
csv_files:list[str] = listdir('State_Summary_Results')
data:dict[str,DataFrame] = {}
df:DataFrame = DataFrame(['State','KH_Vote_Count', 'KH_Vote_Pct', 'DT_Vote_Count', 'DT_Vote_Pct','Pct_Reported'])

In [31]:
for file in csv_files:
    if('FINAL' in file):
        data[file.replace('_Results.csv','').replace('FINAL_','')] = read_csv(f"State_Summary_Results/{file}")

### Merge state summaries into one giant national table

In [32]:
for key,item in data.items():
    item.insert(loc=1,column='State',value=key)
    df:DataFrame = concat([df,item],axis=0)

In [33]:
all_states_summary_df = df.drop([0],axis=1).reset_index(drop=True).iloc[6:]

### Add Nationwide Total Vote Count Columns

In [38]:
total_votes:DataFrame = all_states_summary_df.groupby('Datetime_Extraction').sum(numeric_only=True)

In [ ]:
total_votes['Nationwide_Total_Votes'] = total_votes['KH_Vote_Count']+total_votes['DT_Vote_Count']
total_votes['Additional_Votes_Since_Last_Update'] = total_votes['Nationwide_Total_Votes'].diff()
total_votes['Average_Additional_Votes_Per_Minute'] = total_votes['Additional_Votes_Since_Last_Update'].fillna(0).astype(float)/to_datetime(total_votes.index.to_series()).diff()

UFuncTypeError: ufunc 'divide' cannot use operands with types dtype('float64') and dtype('<m8[ns]')

In [68]:
vote_updates:DataFrame = total_votes.reset_index(drop=False).groupby('Nationwide_Total_Votes').first().reset_index()[['Datetime_Extraction','Nationwide_Total_Votes','KH_Vote_Count','DT_Vote_Count']]

In [69]:
vote_updates['Vote_Change'] = vote_updates['Nationwide_Total_Votes'].diff().fillna(0)

In [ ]:
total_votes[total_votes['Additional_Votes_Since_Last_Update']>0][['KH_Vote_Count','DT_Vote_Count','Nationwide_Total_Votes','Additional_Votes_Since_Last_Update']].to_csv('ANALYSIS/Nationwide_Vote_Count_Updates.csv',index=True)

In [50]:
total_votes[total_votes['Additional_Votes_Since_Last_Update']>0][['KH_Vote_Count','DT_Vote_Count','Nationwide_Total_Votes','Additional_Votes_Since_Last_Update']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1784 entries, 2024-11-05 17:06:07.133829 to 2024-11-07 13:00:18.171214
Data columns (total 4 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   KH_Vote_Count                       1784 non-null   float64
 1   DT_Vote_Count                       1784 non-null   float64
 2   Nationwide_Total_Votes              1784 non-null   float64
 3   Additional_Votes_Since_Last_Update  1784 non-null   float64
dtypes: float64(4)
memory usage: 69.7+ KB
